# ADVANCED TEXT ANALYTICS 2024/2025

## Scope of the project
Starting from a pre-trained model, the goal of the project is to attach a trained [ner](https://spacy.io/api/entityrecognizer) component to the model such that it will recognize labels coming from the medical field. The code is based on the spaCy Python library ([documentation here](https://spacy.io/api/doc)).

To address the ["catastrophic forgetting" problem](https://en.wikipedia.org/wiki/Catastrophic_interference), the trained ner component will be attached to a pre-trained model, the same one used for training the component, so that the output of the model will contain labels that can be assigned either by the original ner or by the trained ner component. Another possible solution could be performing a ["rehease"](https://spacy.io/api/language#rehearse), but in this project it is not explored.

<a id='step0'></a>

### STEP 0: install required libraries and check  for the GPU
Remove the comments to install the libraries required for running this notebook. 

In [ ]:
# %pip install spacy

In [ ]:
# %pip install tensorflow

In [ ]:
# %pip install pandas

In [ ]:
# %pip install spacy-transformers

In [ ]:
# !python -m spacy download en_core_web_trf

# Probably it is required to restart the Jupyter kernel after this instruction

### Installations for NVIDIA GPU

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
# %pip install cupy-wheel

In [1]:
import tensorflow as tf

# Check version of tensorflow and if GPU is available
print(tf.__version__, tf.config.list_physical_devices('GPU'))

2.18.0 []


### Download required models

The models used in this project are:
- `en_core_web_lg`: English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer. Docs: [en_core_web_lg](https://spacy.io/models/en#en_core_web_lg)
- `en_core_web_trf`: English transformer pipeline (Transformer(name=‘roberta-base’, piece_encoder=‘byte-bpe’, stride=104, type=‘roberta’, width=768, window=144, vocab_size=50265)). Components: transformer, tagger, parser, ner, attribute_ruler, lemmatizer. Docs: [en_core_web_trf](https://spacy.io/models/en#en_core_web_trf)

In [ ]:
# !python -m spacy download en_core_web_lg

In [ ]:
# !python -m spacy download en_core_web_trf

<a id='step1'></a>

### STEP 1: prepare training set and dev set

In this project we define the **Annotations** that contains the annotations of the articles, and the **Articles** that contains the articles.

To upload the data use the following logic:
- Store the Annotations Train data inside `./Annotations/Train` folder;
- Store the Annotations Dev data inside `./Annotations/Dev` folder.
- Store the Articles Train data inside `./Articles/Train` folder;
- Store the Articles Dev data inside `./Articles/Dev` folder.

[DocBin](https://spacy.io/api/docbin) is used to store and serialize the Doc objects. The train DocBin will be saved in the `./TrainDocBin/train.spacy` file and the dev DocBin in the `./DevDocBin/dev.spacy` file.

**Note:** go to [step 2](#step2.0) if you already have the train and dev set well formatted.

In [16]:
from pathlib import Path
import os

path = str(Path(os.path.abspath(os.getcwd())).absolute())

# print(path)

# Open training set
with open(path + '/Articles/Train/articles_train.txt','r', encoding='UTF-8') as articlesTrainFile:
  articlesTrain = articlesTrainFile.read().split('\n\n')
  # Remove last empty line if present
  if articlesTrain[len(articlesTrain)-1] == '\n':
    articlesTrain = articlesTrain[:len(articlesTrain)-1]

with open(path + '/Annotations/Train/entities_train.txt','r', encoding='UTF-8') as entitiesTrainFile:
  entitiesTrainFile.readline()
  entitiesTrain = entitiesTrainFile.read().split('\n\n')
  # Remove last empty line if present
  if entitiesTrain[len(entitiesTrain)-1] == '\n':
    entitiesTrain = entitiesTrain[:len(entitiesTrain)-1]

with open(path + '/Articles/Dev/articles_dev.txt','r', encoding='UTF-8') as articlesDevFile:
  articlesDev = articlesDevFile.read().split('\n\n')
  if articlesDev[len(articlesDev)-1] == '\n':
    articlesDev = articlesDev[:len(articlesDev)-1]

with open(path + '/Annotations/Dev/entities_dev.txt','r', encoding='UTF-8') as entitiesDevFile:
  entitiesDevFile.readline()
  entitiesDev = entitiesDevFile.read().split('\n\n')
  if entitiesDev[len(entitiesDev)-1] == '\n':
    entitiesDev = entitiesDev[:len(entitiesDev)-1]

In [17]:
import re

def get_article(text):
  article = re.findall(r'a\|(.*)',text)
  return article[0]

def get_title(text):
  # print(text)
  title = re.findall(r't\|(.*)',text)
  return title[0]

def get_pmid(text):
  pmid = text.split('|', 1)[0]
  pmid = re.sub('\n', '', pmid)
  return pmid

def calc_article_start(text):
  title = re.findall(r't\|(.*)',text)
  return len(title[0]) + 1 # +1 because of space char added between title and abstract

# Articles for train data
train_id = [get_pmid(x) for x in articlesTrain]
train_articles = [get_title(x)+' '+get_article(x) for x in articlesTrain]
train_articles_start_at = [calc_article_start(x) for x in articlesTrain]

# Articles for test data
dev_id = [get_pmid(x) for x in articlesDev]
dev_articles = [get_title(x)+' '+get_article(x) for x in articlesDev]
dev_articles_start_at = [calc_article_start(x) for x in articlesDev]

In [18]:
import pandas as pd

train_df = pd.DataFrame(columns=['article', 'articleStartsAt'])
train_df['pmid'] = train_id
train_df['article'] = train_articles
train_df['articleStartsAt'] = train_articles_start_at

dev_df = pd.DataFrame(columns=['article'])
dev_df['pmid'] = dev_id
dev_df['article'] = dev_articles
dev_df['articleStartsAt'] = dev_articles_start_at

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [19]:
def get_labels(dataframe, text):
  text = text.strip() # Remove possible \n at the start/end of the text
  l = text.split("\n")
  l = [x.split('\t') for x in l]
  labels = []
  index = 0
  for i in l:
    # print(i)
    while dataframe.iloc[index]['pmid'] != i[0]:
      index += 1
      continue

    if i[4] == 'title':
      labels.append((int(i[2]), int(i[3])+1, i[6]))
    elif i[4] == 'abstract':
      # Add shift due to the title length
      labels.append((int(i[2]) + int(dataframe.iloc[index]['articleStartsAt']), int(i[3]) + 1 + int(dataframe.iloc[index]['articleStartsAt']), i[6]))
  return labels

train_labels = [get_labels(train_df, x) for x in entitiesTrain]
dev_labels = [get_labels(dev_df, x) for x in entitiesDev]

print('total train labels: ', len(train_labels), ', total test labels: ' , len(dev_labels))

total train labels:  1566 , total test labels:  40


In [20]:
train_df['labels'] = train_labels
dev_df['labels'] = dev_labels

In [ ]:
train_df.head()

In [ ]:
dev_df.head()

In [21]:
training_data = []
for i, j in zip(train_articles, train_labels):
  training_data.append((i, j))

# print(training_data[0])

dev_data = []
for i, j in zip(dev_articles, dev_labels):
  dev_data.append((i, j))

# print(dev_data[0])

In [22]:
import spacy
import spacy.training

nlp = spacy.load("en_core_web_lg")

<a id='step1.1'></a>

### STEP 1.1: improve Tokenizer

Sometimes a word followed by a punctuation (or viceversa) are not considered as two distinct tokens. To fix this, two different custom tokenizers are implemented:
- the `IMPROVE` tokenizer starts from the default tokenizer and adds some regex pattern to recognize the punctuations and the HTML tags in the prefixes and suffixes of a token
- the `UPDATE` tokenizer creates a new tokenizer and consider the most used punctuation characters and HTML tags as prefixes and suffixes, while for the infix it considers only the regex `[-/+=&]`

Set the variable `WHICH_DOC_TOKENIZER` to one of the possible values to use a particular tokenizer when creating the .spacy files. 

In [ ]:
# Customizing the tokenizer
from spacy.tokenizer import Tokenizer
import re
import string

WHICH_DOC_TOKENIZER = "IMPROVE" # Possible values: DEFAULT, IMPROVE, UPDATE

def improve_tokenizer(nlp):
  my_punct = f"[{re.escape(string.punctuation)}]"
  my_punct = my_punct.replace("<", "")
  my_punct = my_punct.replace(">", "")
  html_tag_pattern = re.compile(r'<[^>]+>')

  all_prefixes_re = spacy.util.compile_prefix_regex(tuple(list(nlp.Defaults.prefixes) + [html_tag_pattern.pattern] + [my_punct]))
  infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)
  suffix_re = spacy.util.compile_suffix_regex(tuple(list(nlp.Defaults.suffixes) + [html_tag_pattern.pattern] + [my_punct]))

  
  return Tokenizer(nlp.vocab,
                  nlp.Defaults.tokenizer_exceptions,
                  prefix_search = all_prefixes_re.search, 
                  infix_finditer = infix_re.finditer,
                  suffix_search = suffix_re.search,
                  token_match=None)

def update_tokenizer(nlp):
  prefix_re = re.compile(r'[.,;:?!(\[\'"</]')
  suffix_re = re.compile(r'[.,;:?!)\]\'">■™]')
  infix_re = re.compile(r'[-/+=&]')

  return Tokenizer(nlp.vocab,
                  prefix_search=prefix_re.search,
                  suffix_search=suffix_re.search,
                  infix_finditer=infix_re.finditer)

if WHICH_DOC_TOKENIZER == "IMPROVE":
  nlp.tokenizer = improve_tokenizer(nlp=nlp)
elif WHICH_DOC_TOKENIZER == "UPDATE":
  nlp.tokenizer = update_tokenizer(nlp=nlp)

In [24]:
from spacy.tokens import DocBin

def save_to_disk(data, dir, filename):
  db = DocBin()
  for text, annotations in data:
    doc = nlp(text)
    # sentence_tokens = []
    # for sent in doc.sents:
    #   sentence_tokens.append([token.text for token in sent])
    # print(sentence_tokens)
    ents = []
    for start, end, label in annotations:
      span = doc.char_span(int(start), int(end), label=label)
      if not span == None:
        ents.append(span)
      else:
        print("none", int(start), int(end), label)
    # print(ents)
    doc.ents = ents
    db.add(doc)

  db.to_disk(os.path.join(dir, filename))

In [ ]:
save_to_disk(training_data, os.path.join(path,'TrainDocBin'), "improved_tokenizer_train.spacy")
save_to_disk(dev_data, os.path.join(path,'DevDocBin'), "improved_tokenizer_dev.spacy")

none 1529 1547 bacteria
none 92 95 gene
none 579 596 anatomical location
none 167 170 DDF
none 359 381 human
none 788 791 DDF
none 806 828 human
none 979 986 human
none 1543 1554 DDF


<a id='step2'></a>

### STEP 2: prepare CUDA and PyTorch

If your PC is already set up correctly, then skip to [step 3](#step3).

#### Check if CUDA is available
The instruction *torch.cuda.is_available()* checks if CUDA is avaiable for running the train on the GPU.
If the answer if false, then it means either PyTorch or CUDA or both of them is not installed.

#### Install PyTorch
To install PyTorch, go to [this link](https://pytorch.org/get-started/locally/), select your preferences (in this case it is important to set a CUDA version as "Compute Platform" so that the code will run on the GPU) and then copy-paste the command into the following cell.

It might be necessary to restart the runtime.

After installing pythorch, *torch.cuda.is_available()* returns true.

In [ ]:
# Check if CUDA is available
import torch

print('is cuda available? ', torch.cuda.is_available())
print(torch.__version__)
!nvidia-smi

<a id='step3'></a>

### STEP 3: train the NER component

#### Generate config.cfg file
Generate the base_config.cfg configuration file that includes all the settings and hyperparameters.
In this project the focus is to train only the ner component.
The train will be optimized for accuracy over efficiency.
Then, save the config to config.cfg file

For this project the training is done with an NVIDIA GeForce 4060 laptop with 8GB of VRAM. 

#### Train en_core_web_trf NER component

In [50]:
!python -m spacy init fill-config ./config/trf/base_config.cfg ./config/trf/config.cfg

✔ Auto-filled config with all values
✔ Saved config
config\trf\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Manuel\ata-project\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# !python -m prodigy spacy-config config.cfg --ner custom_tok_annotations --base-model custom_tokenizer

c:\Users\Manuel\ata-project\Scripts\python.exe: No module named prodigy


In [ ]:
# import spacy
# import spacy.training

# # Load pre-trained model over which a new training for the ner component will be done
# nlp = spacy.load("en_core_web_trf")

Set the variable `WHICH_TRAIN_TOKENIZER` to one of the possible values to use a particular tokenizer when training the model. 

In [ ]:
WHICH_TRAIN_TOKENIZER = "DEFAULT" # Possible values: DEFAULT, IMPROVE, UPDATE

if WHICH_TRAIN_TOKENIZER == "IMPROVE":
  !python -m spacy train ./config/trf/custom-config.cfg --output ./output/trf-train-improve-tokenizer-custom-config --paths.train ./TrainDocBin/improved_tokenizer_train.spacy --paths.dev ./DevDocBin/improved_tokenizer_dev.spacy --code ./improve-tokenizer.py --gpu-id 0
elif WHICH_TRAIN_TOKENIZER == "UPDATE":
  !python -m spacy train ./config/trf/custom-config.cfg --output ./output/trf-train-updated-tokenizer-custom-config --paths.train ./TrainDocBin/updated_tokenizer_train.spacy --paths.dev ./DevDocBin/updated_tokenizer_dev.spacy --code ./update-tokenizer.py --gpu-id 0
elif WHICH_TRAIN_TOKENIZER == "DEFAULT":
  !python -m spacy train ./config/trf/config.cfg --output ./output/trf-all --paths.train ./TrainDocBin/all_train.spacy --paths.dev ./DevDocBin/all_dev.spacy --gpu-id 0

In [ ]:
unique_labels = set()

for example in dev_data:
	entities = example[1]
	for entity in entities:
		entity_label = entity[2]
		unique_labels.add(entity_label)
        
unique_labels_list = list(unique_labels)

print('Entities to be recognised in the provided data: ')
print(unique_labels_list)

### STEP 3.1: evaluate best model

In [ ]:
if WHICH_TRAIN_TOKENIZER == "IMPROVE":
  !spacy evaluate ./output/trf-improve-tokenizer-custom-config/model-best/ ./DevDocBin/improved_tokenizer_dev.spacy --code ./improve-tokenizer.py --gpu-id 0
elif WHICH_TRAIN_TOKENIZER == "UPDATE":
  !spacy evaluate ./output/trf-updated-tokenizer-custom-config/model-best/ ./DevDocBin/updated_tokenizer_dev.spacy --code ./update-tokenizer.py --gpu-id 0
elif WHICH_TRAIN_TOKENIZER == "DEFAULT":
  !spacy evaluate ./output/trf-updated-tokenizer-custom-config/model-best/ ./DevDocBin/updated_tokenizer_dev.spacy --gpu-id 0

<a id='step3'></a>

### STEP 3: attach the trained NER comopnent to the original model
In the following cell, the trained NER component will be integrated into the original en_core_web_trf model. This combination will allow the final model to label words using a set that includes both the original labels and the newly trained ones.

In [ ]:
# Load original model
nlp = spacy.load("en_core_web_trf")
# Load trained model
trained_nlp = spacy.load('./output/model-best')

trained_nlp.replace_listeners("transformer", "ner", ["model.tok2vec"])

nlp.add_pipe(
  "ner",
  name="trained_ner",
  source=trained_nlp,
  before="ner",
)

print(nlp.pipe_names)

In [ ]:
import spacy.displacy


doc = nlp("Doc here")
spacy.displacy.render(doc, style="ent", jupyter=True)

### FINAL: output of best models

| Name | Pipeline | .spacy creation | Params | TOK | NER P | NER R | NER F |
| --- | --- | --- | --- | --- | --- | --- | --- |
| lg-all | en_core_web_lg | Default tokenizer | Default config + Default tokenizer | 99.90 | 81.62 | 72.73 | 76.92 |
| trf-all | en_core_web_trf | Default tokenizer | Default config + Default tokenizer | 99.90 | 83.58 | 80.20 | 81.86 |
| lg-improve-tokenizer | en_core_web_lg | Improved tokenizer | Default config + Default tokenizer | 99.74 | 78.29 | 74.57 | 76.39 |
| trf-improve-tokenizer-custom-config | en_core_web_trf | Improved tokenizer | Custom config + Default tokenizer | 99.74 | 78.89 | 83.97 | 81.35 |
| trf-updated-tokenizer-custom-config | en_core_web_trf | Updated tokenizer | Custom config + Default tokenizer | 90.71 | 80.98 | 81.56 | 81.27 |
| lg-train-improve-tokenizer-custom-config | en_core_web_lg | Improved tokenizer | Custom config + Improved tokenizer | 100.00 | 79.78 | 77.71 | 78.73 |
| trf-train-improve-tokenizer-custom-config | en_core_web_trf | Improved tokenizer | Custom config + Improved tokenizer | 100.00 | 82.47 | 82.54 | 82.51 |